In [1]:
import psycopg2
import duckdb
import pandas as pd
from dotenv import load_dotenv
import os


load_dotenv()

# PostgreSQL-Verbindungsparameter
pg_host = os.getenv('DB_HOST')
pg_dbname = os.getenv('DB_NAME')
pg_user = os.getenv('DB_USER')
pg_password = os.getenv('DB_PASSWORD')

# Verbindung zu PostgreSQL herstellen
pg_conn = psycopg2.connect(
    host=pg_host,
    dbname=pg_dbname,
    user=pg_user,
    password=pg_password
)

# DuckDB-Verbindung (oder neue DuckDB-Datenbank erstellen)
duckdb_conn = duckdb.connect('narrAItive_duckDB.duckdb')

# Funktion zum Laden einer Tabelle von PostgreSQL nach DuckDB
def load_table_to_duckdb(table_name):
    # Daten von PostgreSQL in einen Pandas DataFrame laden
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, pg_conn)
    
    # Daten in DuckDB schreiben
    duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")

# Tabellen, die migriert werden sollen
tables = ["audios", "images", "story", "story_segments", "voices"]

# Tabellen nacheinander migrieren
for table in tables:
    load_table_to_duckdb(table)

# Verbindungen schließen
pg_conn.close()
duckdb_conn.close()

print("Datenbankmigration abgeschlossen!")

Datenbankmigration abgeschlossen!


/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx09wpxhjw1tgbm9575g4/T/ipykernel_21930/557359602.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, pg_conn)
/var/folders/29/9sbs0n5j2dx0

In [2]:
# get tables from duckdb
conn = duckdb.connect('narrAItive_duckDB.duckdb')
cursor = conn.cursor()
cursor.execute("SHOW TABLES;")
print(cursor.fetchall())

[('audios',), ('images',), ('story',), ('story_segments',), ('voices',)]


In [3]:
# get data from duckdb
cursor.execute("SELECT * FROM story;")
print(cursor.fetchall())

[('61888c17-0d9a-4f7d-8c0d-76c03e4df02b', 10, 'Die kleine blonde Mädchen und der grüne Drache', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 9, 51, 21, 347722)), ('5d3f8f1f-ffba-49ce-8e11-c3b7baac55e6', 10, 'Die Ninja Turtles entdecken den Planeten Mars', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 9, 59, 47, 775695)), ('fe18f6a8-04e7-4cba-bb5e-939a2ac1f67d', 10, 'Die Feuerwehrhelden', '3-5', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 11, 32, 13, 850684)), ('1ab9babc-3374-474b-8f0d-27aa1cf7967c', 10, 'Spongebob und Dagobert Duck treffen sich in der Kneipe', '6-8', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 9, 16, 15, 37, 128807)), ('afee59ee-45f8-4bd5-9340-63fb8cf0cfaf', 10, 'Ein Bär im Dschungel', '1-2', 'gpt-3.5-turbo-16k', datetime.datetime(2023, 11, 10, 15, 48, 37, 982716)), ('37f83491-9108-4271-9d38-f60c3d545bb6', 10, 'Peppa Wutz und Dinosaurier spielen Verstecken auf einer Party und essen Brötchen', '3-5', 'gpt-3.5-turbo-16k', datetime.

In [10]:
# create dataframe from table stories
df = conn.table("audios").to_df()
df

,audio_id,segment_id,url,creation_timestamp,creation_model,voice,duration
0,0a015094-d592-407a-9080-a65f7a7cf945,bcd98efc-4826-44c9-b0db-d9f64475fab8,https://narraitive.s3.amazonaws.com/bb3f806a-b...,2023-11-13 13:23:03.944257,tts-1,alloy,NaN
1,b6f92637-137f-42f4-af89-f552be4b9859,cb44254b-0eba-4cb3-a3f5-65762de15922,https://narraitive.s3.amazonaws.com/afee59ee-4...,2023-11-10 16:55:53.655017,tts-1,alloy,NaN
2,21851ba1-17a0-4fa3-a942-09aa13d08705,f7a84f78-e36f-4941-b2e4-dd362a936f13,https://narraitive.s3.amazonaws.com/afee59ee-4...,2023-11-10 16:55:56.818678,tts-1,alloy,NaN
3,02a6d8a0-14b3-4faf-b339-3a29b8b30413,cd02d311-3c1b-4029-b26a-8bb3a2f28a4e,https://narraitive.s3.amazonaws.com/afee59ee-4...,2023-11-10 16:55:59.850299,tts-1,alloy,NaN
4,c4e07824-874f-46ec-9fb9-53b757c264db,bc7cfa6b-267d-414d-88fc-d0e49f1a86d8,https://narraitive.s3.amazonaws.com/afee59ee-4...,2023-11-10 16:56:03.392960,tts-1,alloy,NaN
...,...,...,...,...,...,...,...
325,9307daff-426f-4014-b4a5-95e548dc7369,55de1a25-f16e-4ef9-aef4-9635e991dded,https://narraitive.s3.amazonaws.com/355d1606-5...,2023-12-29 14:32:10.273629,eleven_multilingual_v2,JIvG1qNhaA2YR9O09IoX,NaN
326,4ab008d6-198b-47b7-aeb5-63034b06f13b,30a7b678-ef95-4f8e-8b71-2dae218d87a0,https://narraitive.s3.amazonaws.com/355d1606-5...,2023-12-29 14:32:17.605664,eleven_multilingual_v2,JIvG1qNhaA2YR9O09IoX,NaN
327,07d7179e-5db7-435f-a8f7-c9bb696c64ca,72030076-344d-49e6-afbc-e65f05615212,https://narraitive.s3.amazonaws.com/355d1606-5...,2023-12-29 14:32:22.452415,eleven_multilingual_v2,JIvG1qNhaA2YR9O09IoX,NaN
328,d8e177d3-fa7a-414d-a25d-27426d25dc46,11045991-67ef-4d20-a41d-7d75c8e030e5,https://narraitive.s3.amazonaws.com/355d1606-5...,2023-12-29 14:32:30.738064,eleven_multilingual_v2,JIvG1qNhaA2YR9O09IoX,NaN


In [11]:
def connect_to_duckdb():
    _conn = duckdb.connect('../narrAItive_duckDB.duckdb')
    return _conn

In [18]:
segment_id = "948e3ee3-36ca-4cac-9d07-9442843db767"

with connect_to_duckdb() as conn:
    with conn.cursor() as cursor:
        cursor.execute("""
            SELECT audio_id FROM audios
            WHERE segment_id = '948e3ee3-36ca-4cac-9d07-9442843db767'
        """)
        result = cursor.fetchone()[0]

In [19]:
result

'10c7cec7-b900-4220-bbdb-04e6b7e6f64a'

In [20]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [21]:
import base64
import requests

def get_description_from_image(api_key,
                               image_url,
                               model="gpt-4-vision-preview"):
    # Path to your image
    image_path = image_url

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Please describe the character in this image in about 2 sentences and don't talk about"
                                " the image itself. Just describe the character and start with "
                                "'A toddler, A boy.. / a girl / a man...' Make sure that you describe the colour of "
                                "hair and the potential age of the character."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }
    response = None
    for i_try in range(10):
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        if response.status_code == 200:
            break
    return response.json()["choices"][0]['message']['content']

In [24]:
load_dotenv()
get_description_from_image(os.getenv('OPENAI_API_KEY'), "Thorin.jpg", model="gpt-4o-mini")

'A toddler, a boy with curly blond hair, appears to be around two years old. He is wearing a soft blue sweatshirt and has a thoughtful expression.'